## Learning Realistic Mutations: Bug Creation for Neural Bug Detection
This accompanying notebook is an interactive demo for our contextual mutator. The goal is to inject realistic
bugs into code based on the code context.

## Setup the project
First, we have to locally install the project

In [1]:
!mkdir tmp
!git clone https://github.com/cedricrupb/ctxmutants.git tmp/

Cloning into 'tmp'...
remote: Enumerating objects: 132, done.
remote: Counting objects: 100% (132/132), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 132 (delta 57), reused 127 (delta 52), pack-reused 0
Receiving objects: 100% (132/132), 66.91 KiB | 2.57 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [2]:
%cd tmp

!pip install -r requirements.txt

import ctxmutants

/Users/cedricr/Documents/WorkDir/SCAD/ctxmutants/tmp


## Setup preprocessing
Now, we can setup the preprocessing pipeline for preprocessing code that should be mutated. Here,
we employ a specific syntactic tagger that preserves semantic roles of tokens.

In [3]:
code_example = """

public int additionTest(int a, int b){
    return a + b;
}

"""

from ctxmutants.tokenize import func_tokenize

def tokenize_code(code_snippet):
    try:
        return func_tokenize(code_snippet, lang = "java")
    except Exception:
        return func_tokenize("public class Test{%s}" % code_snippet, lang = "java")


func_to_tokens_types = tokenize_code(code_example)
token_types = next(iter(func_to_tokens_types.values()))

tokens, types = token_types["tokens"], token_types["types"]
print(tokens)

['public', 'int', 'additionTest', '(', 'int', 'a', ',', 'int', 'b', ')', '{', 'return', 'a', '+', 'b', ';', '}']


The `tokenize_code` function successfully (1) identified the functions in the code snippet, (2) produced a tokenized version of each function and (3) computed syntactic types of each token. In the following, we only consider the function discovered first and before we mutate the tokenized code we investigate the token types in the following.

In [4]:
for i, token in enumerate(tokens):
    token_type = types[i]
    print("%s : %s" % (token, token_type.name))

public : KEYWORDS
int : USE_TYPE
additionTest : DEF_FUNC
( : SYNTAX
int : USE_TYPE
a : DEF_VAR
, : SYNTAX
int : USE_TYPE
b : DEF_VAR
) : SYNTAX
{ : SYNTAX
return : KEYWORDS
a : USE_VAR
+ : BOP
b : USE_VAR
; : SYNTAX
} : SYNTAX


Note that each token is assigned a specific syntactic role based on the context it appears in. For example, the identifier `a` is both tagged as a variable definition and variable usage dependent on the program location.

## Setup contextual mutation
Contextual mutators are here implemented within two steps. First, we enumerate potential mutant candidates
by applying a base mutator. Then, we assign a likelihood of seeing a mutant by running a masked language model.
Mutants are produced by sampling according to this likelihood.

In [5]:
import random
from collections import namedtuple

from ctxmutants.mutation import strong_binary_mutation, weak_varmisuse_mutation
from ctxmutants.mutation import weak_binary_mutation
from ctxmutants.mutation import ContextualBatchMutation
from ctxmutants.mutation import mlm_codebert_rerank

MutationCandidate = namedtuple('MutationCandidate', ["tokens", "types", "mask", "location"])

def ctx_mutate(tokens, types, location = None, topK = 5):

    # (1) Select location and base mutator
    base_mutators = {"USE_VAR": weak_varmisuse_mutation, "BOP": strong_binary_mutation} # Option: replace strong_binary_mutation by weak_binary_mutation
    if location is None: 
        candidate_locs = [i for i, t in enumerate(types) if t.name in base_mutators]
        assert len(candidate_locs) > 0
        location = random.choice(candidate_locs)
    else:
        assert types[location].name in base_mutators

    base_mutator = base_mutators[types[location].name]

    # (2) Setup mutator with base_mutator and use CodeBert for contextual mutation
    mutator = ContextualBatchMutation(base_mutator, mlm_codebert_rerank, lang = "java")
    mutant_dist = mutator(
        [MutationCandidate(tokens, types, None, location)]
    )
    mutant_dist = mutant_dist[0]

    # (3) We report the topK mutants
    topK_mutants = sorted(mutant_dist.items(), key = lambda x: x[1], reverse = True)
    topK_mutants = topK_mutants[:topK]

    output = []

    for mutant, prob in topK_mutants:
        mutant_tokens = list(tokens)
        mutant_tokens[location] = mutant
        mutant_code = " ".join(mutant_tokens)

        output.append({
            "code": mutant_code,
            "location": location,
            "before" : tokens[location],
            "after": mutant,
            "score" : prob
        })


    return output


ctx_mutate(tokens, types, location = 13)


[{'code': 'public int additionTest ( int a , int b ) { return a - b ; }',
  'location': 13,
  'before': '+',
  'after': '-',
  'score': 0.8357382491308553},
 {'code': 'public int additionTest ( int a , int b ) { return a * b ; }',
  'location': 13,
  'before': '+',
  'after': '*',
  'score': 0.08246741274092674},
 {'code': 'public int additionTest ( int a , int b ) { return a % b ; }',
  'location': 13,
  'before': '+',
  'after': '%',
  'score': 0.07746933939678168},
 {'code': 'public int additionTest ( int a , int b ) { return a / b ; }',
  'location': 13,
  'before': '+',
  'after': '/',
  'score': 0.004324998731436322}]

We see that replacing the addition with a subtraction is the most likely mutation. To see why it is important to restrict the mutation
candidates to syntactically fitting candidates, please replace `strong_binary_mutation` by `weak_binary_mutation` and run the code again. What do you observe?

## Playground
Now, it is your turn to play with contextual mutations. Use the variable `source_code` to process your own code.
After running the next cell, choose a potential mutation location or leave it to the algorithm.

In [6]:
source_code = """

public void test(Object input) {
    if(input == null) {
        return;
    }
    handle(input);
}

"""

tokens_types  = next(iter(tokenize_code(source_code).values()))
tokens, types = tokens_types["tokens"], tokens_types["types"]

print("Tokens:")
print(tokens)
print()
print("Available mutation locaions:")

for i, token in enumerate(tokens):
    type = types[i]
    if type.name in ["USE_VAR", "BOP"]:
        token_ctx = " ".join(tokens[i-3:i+3])
        print("Loc %d | Token: %s  | Type: %s | Ctx: %s" % (i, token, type.name, token_ctx))

Tokens:
['public', 'void', 'test', '(', 'Object', 'input', ')', '{', 'if', '(', 'input', '==', 'null', ')', '{', 'return', ';', '}', 'handle', '(', 'input', ')', ';', '}']

Available mutation locaions:
Loc 10 | Token: input  | Type: USE_VAR | Ctx: { if ( input == null
Loc 11 | Token: ==  | Type: BOP | Ctx: if ( input == null )
Loc 20 | Token: input  | Type: USE_VAR | Ctx: } handle ( input ) ;


Choose a mutant location such that the contextual mutator can produce some mutation candidates.

In [7]:
mutant_location = 11 # Change here (Set to None if random)

mutations = ctx_mutate(tokens, types, location = mutant_location)

for mutation in mutations:
    print(mutation)
    print("----")


{'code': 'public void test ( Object input ) { if ( input != null ) { return ; } handle ( input ) ; }', 'location': 11, 'before': '==', 'after': '!=', 'score': 0.9630580979475805}
----
{'code': 'public void test ( Object input ) { if ( input <= null ) { return ; } handle ( input ) ; }', 'location': 11, 'before': '==', 'after': '<=', 'score': 0.02740058623154283}
----
{'code': 'public void test ( Object input ) { if ( input < null ) { return ; } handle ( input ) ; }', 'location': 11, 'before': '==', 'after': '<', 'score': 0.008276679278432718}
----
{'code': 'public void test ( Object input ) { if ( input >= null ) { return ; } handle ( input ) ; }', 'location': 11, 'before': '==', 'after': '>=', 'score': 0.0006765373083551387}
----
{'code': 'public void test ( Object input ) { if ( input > null ) { return ; } handle ( input ) ; }', 'location': 11, 'before': '==', 'after': '>', 'score': 0.0005880992340889765}
----


### Comparison between mutation types
This section also belongs to the playground and, hence, uses the same code and location defined in the previous section.
Here, we want to highlight the difference between different mutator types used in neural bug detection, mutation testing and in our paper.

In [9]:
# For sampling a mutation
def sample_mutation(mutation_dist):
    random_selection = random.random()
    cumsum = 0.0

    for key, prob in sorted(mutation_dist.items(), key = lambda x: x[1], reverse=True):
        if cumsum + prob >= random_selection: return key
        cumsum += prob
    
    return key # Because of numerical instabilities sum(probs) smaller 1.0


assert types[mutant_location].name == "BOP", "Only all three mutation types are supported for BOP"

# Loose mutant
loose_mutation_dist = weak_binary_mutation(tokens, mutant_location)
loose_mutant = sample_mutation(loose_mutation_dist)

# Strict mutant
strong_mutation_dist = strong_binary_mutation(tokens, mutant_location)
strong_mutant = sample_mutation(strong_mutation_dist)

# Contextual mutant

ctx_mutation_dist = {m["after"] : m["score"] for m in mutations}
ctx_mutant = sample_mutation(ctx_mutation_dist)

for mutant_type, mutant in [("Loose mutation (as used in neural bug detection)", loose_mutant) , ("Strict mutation (as used in mutation testing)", strong_mutant), ("Contextual mutation", ctx_mutant)]:
    print(mutant_type)
    print("----------------------------------------------------------------")

    mutate_tokens = list(tokens)
    mutate_tokens[mutant_location] = mutant
    mutate_code = " ".join(mutate_tokens)

    print(mutate_code)
    print()


Loose mutation (as used in neural bug detection)
----------------------------------------------------------------
public void test ( Object input ) { if ( input | null ) { return ; } handle ( input ) ; }

Strict mutation (as used in mutation testing)
----------------------------------------------------------------
public void test ( Object input ) { if ( input >= null ) { return ; } handle ( input ) ; }

Contextual mutation
----------------------------------------------------------------
public void test ( Object input ) { if ( input != null ) { return ; } handle ( input ) ; }

